# Ottenimento prossimi match
Lo script deve permettere di prendere i prossimi 10 match. 
Verrà fatta una predizione su questi match, prima deve essere calcolata la media di ciascun match che è data dai 5 match precedenti.

In [1]:
from analysis import MatchAnalysis
import pandas as pd
import util_strings as utils
from ranking import Ranking
from football_predictions import FootballPredictions

tfidf = True

In [2]:
championship = pd.read_csv(utils.championship_actual_season, index_col=0)
season = utils.championship_actual_season.replace('.csv', '')[-9:]

championship['date'] = pd.to_datetime(championship['date'], format='%Y-%m-%d')

championship.result = championship.result.astype(str)
new_matches = championship[championship.result == 'nan']

In [3]:
matches = new_matches[:10][['team1', 'team2', 'date', 'result']]
matches['season'] = season
matches.rename(columns= {'team1': 'home', 'team2': 'away'}, inplace=True)
matches.reset_index(drop=True, inplace=True)
matches.home = matches.home.str.lower()
matches.away = matches.away.str.lower()

matches.to_csv(utils.next_matches)

## Ottenimento testo FootballPrediction
Scaricamento del testo di ciascuna partita

In [4]:
fp = FootballPredictions(matches=matches)
fp.update_originalNotation()
fp.get_urls()
fp.get_predictions(utils.next_matches, False)
fp.df['season'] = season
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates(utils.next_matches)

100%|██████████| 13/13 [00:26<00:00,  2.07s/it]


In [5]:
from my_tokenizer import MyTokenizer

mt = MyTokenizer(fp.df)
mt.feature_normalization()
mt.clean_text()
mt.open_vectorizer(utils.TfidfVectorizer if tfidf else utils.CountVectorizer)

In [6]:
import pickle

dt = pickle.load(open(utils.classificatorTfIdf if tfidf else utils.classificatorCV_LR, 'rb'))
predictions = dt.predict(mt.vectorizer.transform(mt.cleaned_corpus))

Aggiungo le predizioni al dataset dei match

In [7]:
cont = 0
for i, match in matches.iterrows():
    matches.at[i, 'prediction'] = predictions[cont]
    cont += 1

## Calcolo delle medie
Per ciascun match devo calcolare le statistiche date dalle medie dei 5 match precedenti e poi devo ottenere la predizione di football_prediction.
Sostituisco inoltre i codici delle squadre

Per calcolare le medie devo prendere tutti i match di ciascuna squadra

In [8]:
ranking = Ranking('SerieA', '2022-2023')
ranking.read_matches(seasons=6, path=utils.ranking)

ma = MatchAnalysis()
ma.set_ranking(ranking=ranking)
#leggo tutti i match (doppi perché ci sono le statistiche x ogni squadra)
ma.read_matches(utils.merged_statistics)
#creo una lista di oggetti (lista di squadre)
ma.create_team_dataset() 

In [9]:
import json
with open(utils.teams_codes, 'rb') as json_file:
    id_set = json.load(json_file)

In [10]:
matches['date'] = pd.to_datetime(matches['date'], format='%Y-%m-%d') 
float_features_and_avg = ['goals','total_shots','shots_on_target','goals_on_penalty','total_penalties','percentage_possession','fouls','yellow_cards','red_cards']

import numpy as np
for f in float_features_and_avg:
    matches[f] = np.nan

matches[float_features_and_avg] = matches[float_features_and_avg].astype(float)

label = {'N': 0, 'V': 1, 'P':2}

ma.diff_dataset = matches.copy()
for i, match in ma.diff_dataset.iterrows(): 
    match.season = '2022-2023'
    avgH, c = ma.get_team_by_name(match.home).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    avgA, c = ma.get_team_by_name(match.away).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    
    ma.get_ranks(match, i)

    for col in float_features_and_avg:
        diff = avgH[col] - avgA[col]
        ma.diff_dataset.at[i, col] = diff
        
    ma.diff_dataset.at[i, 'home'] = id_set[match.home]
    ma.diff_dataset.at[i, 'away'] = id_set[match.away]

ma.diff_dataset.drop(columns=['result', 'date', 'season'], inplace=True)

for i, m in ma.diff_dataset.iterrows():
    ma.diff_dataset.at[i, 'rank'] = m.rank_h - m.rank_a
ma.diff_dataset.drop(columns=['rank_h', 'rank_a'], inplace=True)

In [11]:
#ma.diff_dataset.home = ma.diff_dataset.home.astype(float)
#ma.diff_dataset.away = ma.diff_dataset.away.astype(float)

In [12]:
ma.diff_dataset
path_model = utils.final_classificator_Tfidf if tfidf else utils.final_classificator_CV
#path_model = 'ML model/Tfidf_SVC.model'
dt = pickle.load(open(path_model, 'rb'))
predictions = dt.predict(ma.diff_dataset)

In [13]:
ma.diff_dataset

,home,away,prediction,goals,total_shots,shots_on_target,goals_on_penalty,total_penalties,percentage_possession,fouls,yellow_cards,red_cards,rank
0,4,11,1.0,-0.4,-3.8,-0.4,-0.2,-0.2,0.0,-3.2,-1.0,0.0,-9.0
1,8,14,1.0,0.4,3.4,0.2,0.0,-0.2,5.4,0.0,0.6,0.0,2.0
2,12,18,1.0,0.4,6.2,0.8,0.2,0.2,6.2,-5.6,-1.2,0.0,8.0
3,9,1,1.0,0.2,1.2,-0.2,-0.2,0.0,-1.2,5.2,0.8,0.6,11.0
4,10,28,1.0,1.0,2.8,3.2,0.0,0.0,6.0,-5.8,-0.6,-0.2,12.0
5,25,22,1.0,0.4,-2.0,0.0,0.0,0.2,-5.4,-0.2,0.4,0.0,2.0
6,0,5,1.0,0.8,-1.8,0.4,0.4,0.0,-10.2,-1.4,0.4,-0.2,1.0
7,17,27,1.0,-1.2,-0.4,0.4,-0.2,0.0,-4.2,-3.4,-0.6,0.0,11.0
8,16,26,1.0,0.2,1.4,2.0,0.0,0.0,-5.4,-1.6,0.4,0.0,12.0
9,24,19,1.0,-2.0,-3.2,-1.2,0.0,0.0,3.6,5.4,1.2,0.0,-5.0


In [14]:
df_predictions = pd.DataFrame()
for i, match in matches.iterrows():
    match['prediction'] = predictions[i]
    df_predictions = df_predictions.append(match[['home','away','prediction']])

In [15]:
df_predictions

,home,away,prediction
0,empoli,milan,2.0
1,inter,roma,1.0
2,napoli,torino,1.0
3,juventus,bologna,1.0
4,lazio,spezia,1.0
5,lecce,cremonese,1.0
6,atalanta,fiorentina,1.0
7,sassuolo,salernitana,1.0
8,sampdoria,monza,1.0
9,hellas verona,udinese,2.0


In [16]:
res = [2,1,2,2,1,2,1,2,1,1,2,0,0,0,1,1,2,2,1,1,2,0,1,0,1,2,1,0,1,0,2,2,1,1,0,1,1,1,2,2]
res.reverse()

In [17]:
i = 0
count = 0

for d in df_predictions.prediction:
    if d == res[i]:
        count+=1
    i+=1
    
print(count, "su", len(df_predictions))

19 su 40
